In [ ]:
import geopandas as gpd
import pandas as pd
from pyproj import Proj, CRS, Transformer
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
from folium import plugins
import branca
import seaborn as sns
plt.rcParams['figure.figsize'] = [8,8]



In [ ]:
gb_ne = CRS('epsg:29901')
ll = CRS('epsg:4326')
tr = Transformer.from_crs(gb_ne, ll)

In [ ]:
df = pd.read_csv("/home/rebecca/Documents/soil/csv/regionalsoilssaquaregiahdl.csv")
gdf = gpd.GeoDataFrame(df)
gdf

In [ ]:
def map_row(row):
  latlng = tr.transform(row["Easting"], row["Northing"])
  return Point(latlng[1], latlng[0])

#def lat_lng(data):
#  lat,lng = tr.transform(data["Easting"], data["Northing"])
#  return (lat, lng)

In [ ]:
geometry = pd.DataFrame(gdf.apply(map_row, axis=1), columns = ["geometry"])

#latlng = pd.DataFrame(lat_lng(gdf), index = ["lat", "lng"]).T

In [ ]:
#aquaRD = pd.concat([gdf, latlng, geometry], axis=1)
aquaRD = pd.concat([gdf, geometry], axis=1)
aquaRD.set_geometry("geometry", inplace=True)
aquaRD.head()

In [ ]:
#aquaRD_df = aquaRD[["Sample","P","Se","Cu","Mo","Mn","Ca","K","Zn","Mg","Pb","Cd","As","lat","lng","geometry"]]
aquaRD_df = aquaRD[["Sample_Number","P","Se","Cu","Mo","Mn","Ca","K","Zn","Mg", "Na","Pb","Cd","As", "Ni","Cr", "Hg","geometry"]]
#aquaRD_df

In [ ]:
xrf = pd.read_csv("/home/rebecca/Documents/soil/csv/regionalsoilsaxrf.csv")
xrf_df= pd.DataFrame(xrf[["Sample","pH", "LOI"]])

In [ ]:
aquaRD_df=aquaRD_df.rename(columns={'Sample_Number':'Sample'})
ard_a = pd.merge(aquaRD_df, xrf_df, on='Sample', how='inner')

In [ ]:
#def som(row):
#    return round(100-row["LOI"],2)

#ard_a['%OM'] = ard_a.apply(som,axis=1)
#ard_a

ard_a = ard_a.rename(columns={'LOI':'%OM'})
ard_a

In [ ]:
ard_a.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

In [ ]:
a = pd.to_numeric(ard_a["Mn"], errors='coerce')
idx=a.isna()
ard_a[idx]

In [ ]:
ard_a["Ca"]=ard_a["Ca"].replace(">15", 20)
ard_a["Mn"]=ard_a["Mn"].replace("--", 0)
ard_a["Mn"]=ard_a["Mn"].replace(">10000", 10100)

In [ ]:
ard_A=ard_a.copy()

In [ ]:
ard_A.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

In [ ]:
ard_A

In [ ]:
sum(i>3 for i in pd.to_numeric(ard_A["Se"]))

In [ ]:
ard_A.to_csv("deep_soil_aqua.csv", index=False)

In [ ]:
coordinates = pd.DataFrame()
coordinates['lat']=ard_A.geometry.y
coordinates['lng']=ard_A.geometry.x

In [ ]:
def create_nimap():
    nimap = folium.Map(
        location=[54.637521, -6.485935],
        zoom_start=9
    )
    return nimap

In [ ]:
ni_map = create_nimap()
plugins.MarkerCluster(coordinates[['lat','lng']]).add_to(ni_map)

fig = folium.Figure(width=1500, height=1000)
ni_map.add_to(fig)

In [ ]:
import branca 
ni_map = create_nimap()
fig = folium.Figure(width=1300, height=800)
colormap = branca.colormap.linear.viridis.to_step(index=range(0,9,1))

pH=ard_A.explore("pH", m=ni_map, cmap = colormap, marker_kwds={"radius": 3}, name = "Results")

folium.TileLayer('stamenterrain').add_to(pH)
folium.TileLayer('cartodbpositron').add_to(pH)
folium.TileLayer('cartodbdark_matter').add_to(pH)
folium.LayerControl().add_to(pH)


pH.add_to(fig)


In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1300, height=1000)
colormap = branca.colormap.linear.viridis.to_step(index=range(0,110,20))
colormap.caption="%OM"
                                                  
OM=ard_A.explore("%OM", cmap=colormap, m=ni_map, marker_kwds={"radius": 3}, name="Results")

folium.TileLayer('stamenterrain').add_to(OM)
folium.TileLayer('cartodbpositron').add_to(OM)
folium.TileLayer('cartodbdark_matter').add_to(OM)
folium.LayerControl().add_to(OM)
OM.add_to(fig)

In [ ]:
ard_A["pH"].describe().to_frame()

In [ ]:
print()
print("Low pH AND low Organic matter")
print()
ll=ard_A.loc[(ard_A['pH']<= 3.5) & (ard_a['%OM']<20)]
ll

In [ ]:
len(ll)

In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1200, height=1000)

ll_coord = pd.DataFrame()
ll_coord['lat']=ll.geometry.y
ll_coord['lng']=ll.geometry.x

plugins.MarkerCluster(ll_coord[['lat','lng']]).add_to(ni_map)
ni_map.add_to(fig)

In [ ]:
basic_cations=ard_A[["Ca","Mg","K","Na"]].copy().astype(float)

def basic_conversion(data):
    Ca_=(data["Ca"])/200
    Mg_=(data["Mg"])/120
    K_=(data["K"])/390
    Na_=(data["Na"])/23
    return [Ca_, Mg_, K_, Na_]

basic_meq = pd.DataFrame(basic_conversion(basic_cations), index = ["Ca","Mg","K","Na"]).T
#basic_meq

In [ ]:
acid_cations=pd.DataFrame(ard_A["pH"])

In [ ]:
acid_cations["pH_eq"]=acid_cations["pH"].apply(lambda ph: 12 * (7 - ph) if ph < 7 else 0)
#acid_cations

In [ ]:
CEC_df = pd.concat([basic_meq, acid_cations["pH_eq"]], axis=1)
CEC_df["CEC"]=round(CEC_df.sum(axis=1),2)

#CEC_df

In [ ]:
CEC_saturation = CEC_df["CEC"].copy().to_frame()

CEC_saturation["Acid%"] = round((CEC_df["pH_eq"]/CEC_df["CEC"])*100,2)
CEC_saturation["Base%"] = round(((CEC_df["Ca"]+CEC_df["Mg"]+CEC_df["K"]+CEC_df["Na"])/CEC_df["CEC"])*100,2)
CEC_saturation["Ca%"] = round((CEC_df["Ca"]/CEC_df["CEC"])*100,2)
CEC_saturation["Mg%"] = round((CEC_df["Mg"]/CEC_df["CEC"])*100,2)
CEC_saturation["K%"] = round((CEC_df["K"]/CEC_df["CEC"])*100,2)
CEC_saturation["Na%"] = round((CEC_df["Na"]/CEC_df["CEC"])*100,2)

In [ ]:
#CEC_saturation

In [ ]:
def soil_texture(data):
    if data <=9:
        return "Sand"
    elif data >10 and data <=17:
        return "Loam"
    elif data >17 and data <=25:
        return "Silt"
    elif data >25 and data<=50:
        return "Clay"
    else:
        return "Organic Soil"

In [ ]:
CEC_saturation["Type"] = CEC_saturation["CEC"].apply(soil_texture)

In [ ]:
CEC_saturation

In [ ]:
ardA_type = pd.concat([ard_A, CEC_saturation["Type"]], axis=1)
ardA_type.to_csv("deep_soil_CEC.csv", index=False)
ardA_type.drop(["Sample","geometry"], axis=1, inplace=True)
ardA_type

In [ ]:
soil_type=CEC_saturation.copy().groupby(["Type"])["Type"].count().nlargest().to_frame()
soil_type

In [ ]:
plt.pie(soil_type["Type"], labels=soil_type.index, radius=1.5);

In [ ]:
CEC_loc = gpd.GeoDataFrame(pd.concat([CEC_saturation, ard_A.geometry], axis =1))
CEC_loc.head()

In [ ]:
soil_type = CEC_loc.copy()
soil_type['lat']=soil_type.geometry.y
soil_type['lng']=soil_type.geometry.x

Org_soil = soil_type[(soil_type['Type']=="Organic Soil")]
Clay_soil = soil_type[(soil_type['Type']=="Clay")]
Silt_soil = soil_type[(soil_type['Type']=="Silt")]
Loam_soil = soil_type[(soil_type['Type']=="Loam")]
Sand_soil = soil_type[(soil_type['Type']=="Sand")]


In [ ]:
P=gpd.GeoDataFrame(pd.concat([(pd.to_numeric(ard_A["P"], errors='coerce')),ard_A["pH"], ard_A["geometry"]], axis=1))
P

In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1200, height=1000)
colormap = branca.colormap.linear.Accent_06.to_step(index=[0,1500,9000])

P_map=P.explore("P", cmap = colormap, m=ni_map, marker_kwds={"radius": 3})
P_map.add_to(fig)